### Therapeutics Data Commons: ADMET
### Show Results


In [1]:
from utils import load_json
from tdc_bechmarks_july2025 import benchmark_results, get_sota_df, merge_dfs, create_results_df, create_results_df, merge_dfs

# Every value of MolE is from the paper, they have not submitted to the leaderboard nor have a GitHub with results
mole_results_dict = {k: v['leaderboard']['MolE'] for k, v in benchmark_results.items() if 'MolE' in v['leaderboard']}
minimol_results_dict = {k: v['leaderboard']['MiniMol'] for k, v in benchmark_results.items() if 'MiniMol' in v['leaderboard']}

tabpfn_results_dict = load_json("results_tabpfn.json")
xgboost_results_dict = load_json("results_xgboost.json")

sota_df = get_sota_df()
tabpfn_df = create_results_df(tabpfn_results_dict, "TabPFN ")

minimol_df = create_results_df(minimol_results_dict, "MiniMol ")
mole_df = create_results_df(mole_results_dict, "MolE ")
xgb_df = create_results_df(xgboost_results_dict, "XGBoost ")

results = merge_dfs([sota_df, minimol_df, mole_df, tabpfn_df, xgb_df])

print(f"MiniMol mean rank:     {results['MiniMol position'].mean():.2f}")
print(f"MolE mean rank:        {results['MolE position'].mean():.2f}")
print(f"TabPFN mean rank:     {results['TabPFN position'].mean():.2f}")
print(f"XGBoost mean rank:    {results['XGBoost position'].mean():.2f}")
print()

print(f"TabPFN on 1º:         {(results['TabPFN position'] == 1).sum()} times")
print(f"TabPFN > MiniMol:     {(results['TabPFN position'] <= results['MiniMol position']).sum()} times")
print(f"TabPFN > MolE:       {(results['TabPFN position'] <= results['MolE position']).sum()} times")
print(f"TabPFN > XGBoost:    {(results['TabPFN position'] < results['XGBoost position']).sum()} times")

results

MiniMol mean rank:     4.32
MolE mean rank:        8.95
TabPFN mean rank:     7.00
XGBoost mean rank:    12.27

TabPFN on 1º:         3 times
TabPFN > MiniMol:     9 times
TabPFN > MolE:       14 times
TabPFN > XGBoost:    17 times


,metric,task,SOTA result,MiniMol result,MiniMol position,MolE result,MolE position,TabPFN result,TabPFN position,XGBoost result,XGBoost position
dataset,,,,,,,,,,,
caco2_wang,MAE (↓),regression,0.256 ± 0.006,0.350 ± 0.018,16,0.310 ± 0.010,9,0.276 ± 0.001,3,0.315 ± 0.000,10
bioavailability_ma,AUROC (↑),classification,0.942 ± 0.002,0.942 ± 0.002,1,0.654 ± 0.028,21,0.748 ± 0.009,21,0.741 ± 0.000,21
lipophilicity_astrazeneca,MAE (↓),regression,0.456 ± 0.008,0.456 ± 0.008,1,0.469 ± 0.009,3,0.503 ± 0.002,6,0.575 ± 0.000,17
solubility_aqsoldb,MAE (↓),regression,0.741 ± 0.013,0.741 ± 0.013,1,0.792 ± 0.005,6,0.808 ± 0.005,9,0.817 ± 0.000,9
hia_hou,AUROC (↑),classification,0.993 ± 0.005,0.993 ± 0.005,1,0.963 ± 0.019,14,0.986 ± 0.000,5,0.976 ± 0.000,10
pgp_broccatelli,AUROC (↑),classification,0.942 ± 0.002,0.942 ± 0.002,1,0.915 ± 0.005,9,0.939 ± 0.002,2,0.919 ± 0.000,8
bbb_martins,AUROC (↑),classification,0.924 ± 0.003,0.924 ± 0.003,1,0.903 ± 0.005,11,0.926 ± 0.002,1,0.876 ± 0.000,18
ppbr_az,MAE (↓),regression,7.440 ± 0.024,7.696 ± 0.125,4,8.073 ± 0.335,8,7.033 ± 0.053,1,8.348 ± 0.000,11
vdss_lombardo,Spearman (↑),regression,0.713 ± 0.007,0.535 ± 0.027,10,0.654 ± 0.031,3,0.697 ± 0.004,3,0.526 ± 0.000,11


### Check how many parameters the model has

In [ ]:
import torch

ckpt_path = "tabpfn_ckpts/"
ckpts = {'Regressor': 'tabpfn-v2-regressor.ckpt',
         'Classifier': 'tabpfn-v2-classifier-finetuned-zk73skhh.ckpt'}

for task, ckpt_name in ckpts.items():
    ckpt = torch.load(ckpt_path + ckpt_name, map_location='cpu')

    state_dict = ckpt.get("state_dict", ckpt)
    total_params = sum(p.numel() for p in state_dict.values() if isinstance(p, torch.Tensor))

    print(f"{task} total params: {total_params:,}")

Regressor total params: 11,091,865
Classifier total params: 7,244,554
